In [9]:
import math
import copy
import os
import numpy as np
import pandas as pd

In [16]:
clusterMapData = pd.read_csv('./training_data/cluster_map/cluster_map', sep='\t', names=['region_hash', 'region_id'])
print(clusterMapData.head())

                        region_hash  region_id
0  90c5a34f06ac86aee0fd70e2adce7d8a          1
1  f2c8c4bb99e6377d21de71275afd6cd2          2
2  58c7a4888306d8ff3a641d1c0feccbe3          3
3  b26a240205c852804ff8758628c0a86a          4
4  4b9e4cf2fbdc8281b8a1f9f12b80ce4d          5


In [24]:
orderDataPath = './training_data/order_data/'

filesToExplore = []
for file in os.listdir(orderDataPath):
    if file.startswith('order_data'):
        filesToExplore.append(file)
        print(f"{file} read")
    else:
        continue

print(f"{len(filesToExplore)} files read")
dataTypes = {'order_id':'str', 'driver_id':'str', 'passenger_id':'str', 'start_region_hash':'str', 'dest_region_hash':'str', 'price':'double', 'time':'str'}
orderData = []
for files in filesToExplore:
    fileRead = pd.read_csv(orderDataPath + files, sep='\t', names=['order_id', 'driver_id', 'passenger_id', 'start_region_hash', 'dest_region_hash', 'price', 'time'],dtype=dataTypes)
    orderData.append(fileRead)

orderData = pd.concat(orderData, ignore_index=True)

print(orderData.head())

order_data_2016-01-01 read
order_data_2016-01-02 read
order_data_2016-01-03 read
order_data_2016-01-04 read
order_data_2016-01-05 read
order_data_2016-01-06 read
order_data_2016-01-07 read
order_data_2016-01-08 read
order_data_2016-01-09 read
order_data_2016-01-10 read
order_data_2016-01-11 read
order_data_2016-01-12 read
order_data_2016-01-13 read
order_data_2016-01-14 read
order_data_2016-01-15 read
order_data_2016-01-16 read
order_data_2016-01-17 read
order_data_2016-01-18 read
order_data_2016-01-19 read
order_data_2016-01-20 read
order_data_2016-01-21 read
21 files read
                           order_id                         driver_id  \
0  97ebd0c6680f7c0535dbfdead6e51b4b  dd65fa250fca2833a3a8c16d2cf0457c   
1  92c3ac9251cc9b5aab90b114a1e363be  c077e0297639edcb1df6189e8cda2c3d   
2  abeefc3e2aec952468e2fd42a1649640  86dbc1b68de435957c61b5a523854b69   
3  cb31d0be64cda3cc66b46617bf49a05c  4fadfa6eeaa694742de036dddf02b0c4   
4  139d492189ae5a933122c098f63252b3                   